In [209]:
import pandas as pd
import numpy as np
import json
import csv
import torch
import torch.nn as nn
import sklearn
import torch.nn.functional as FF
import random

## Load Data

In [210]:
# Load Public
dataset = 'Task_2_data_public'

with open(f"data/{dataset}/intervention_0.json", mode="r") as F:
    data_0 = json.load(fp=F)
with open(f"data/{dataset}/intervention_1.json", mode="r") as F:
    data_1 = json.load(fp=F)
with open(f"data/{dataset}/intervention_2.json", mode="r") as F:
    data_2 = json.load(fp=F)
with open(f"data/{dataset}/intervention_3.json", mode="r") as F:
    data_3 = json.load(fp=F)
with open(f"data/{dataset}/intervention_4.json", mode="r") as F:
    data_4 = json.load(fp=F)

In [211]:
# Load Local
dataset = 'Task_2_data_local_dev'

with open(f"data/{dataset}/intervention_0.json", mode="r") as F:
    local_0 = json.load(fp=F)
with open(f"data/{dataset}/intervention_1.json", mode="r") as F:
    local_1 = json.load(fp=F)
with open(f"data/{dataset}/intervention_2.json", mode="r") as F:
    local_2 = json.load(fp=F)
with open(f"data/{dataset}/intervention_3.json", mode="r") as F:
    local_3 = json.load(fp=F)
with open(f"data/{dataset}/intervention_4.json", mode="r") as F:
    local_4 = json.load(fp=F)

label = np.load("data/Task_2_data_local_dev/cate_estimate.npy")

In [212]:
# Load Private

dataset = 'Task_2_data_private'

with open(f"data/{dataset}/intervention_0.json", mode="r") as F:
    private_0 = json.load(fp=F)
with open(f"data/{dataset}/intervention_1.json", mode="r") as F:
    private_1 = json.load(fp=F)
with open(f"data/{dataset}/intervention_2.json", mode="r") as F:
    private_2 = json.load(fp=F)
with open(f"data/{dataset}/intervention_3.json", mode="r") as F:
    private_3 = json.load(fp=F)
with open(f"data/{dataset}/intervention_4.json", mode="r") as F:
    private_4 = json.load(fp=F)

# Process

In [213]:
def process_data(dataset):
    processed = []
    for i in range(len(dataset)):
        processed.append(
            [
                [[i[0]]+i[1:51]+[i[int(i[0])]] for i in dataset[i]['conditioning']],
                dataset[i]['intervention'][0][0],
                dataset[i]['reference'][0][0],
                dataset[i]['effect_mask'][2].index(True),
            ]
        )
    return processed

processed_0 = process_data(data_0)
processed_1 = process_data(data_1)
processed_2 = process_data(data_2)
processed_3 = process_data(data_3)
processed_4 = process_data(data_4)

local_0 = process_data(local_0)
local_1 = process_data(local_1)
local_2 = process_data(local_2)
local_3 = process_data(local_3)
local_4 = process_data(local_4)

private_0 = process_data(private_0)
private_1 = process_data(private_1)
private_2 = process_data(private_2)
private_3 = process_data(private_3)
private_4 = process_data(private_4)

## Model Definition

In [214]:
# 模型定义
class MyRNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.GRU(batch_first=True, input_size=200+50+1, hidden_size=256)
        self.embedding = nn.Embedding(50, 200)
        self.fc = nn.Linear(256, 128)
        self.fc2 = nn.Linear(128, 50)
        self.dropout = nn.Dropout(0.2)
        # initialize
        # for name, param in self.named_parameters():
        #     if 'weight' in name:
        #         nn.init.xavier_normal_(param)

    def forward(self, input):
        kc = input[:, :, 0].long()
        state = input[:, :, 1:51]
        ans = input[:, :, 50:51]
        knows = self.embedding(kc)

        input = torch.cat([
            knows, state, ans
        ], dim=-1)

        out, h = self.model(input)
        out = self.dropout(out)
        return self.fc2(
                FF.relu(
                    self.fc(out)
                )
            )
    
    def predict(self, input, h=None):
        if h is None:
            kc = input[:, 0].long()
            state = input[:, 1:51]
            ans = input[:, 50:51]
            knows = self.embedding(kc)
            input = torch.cat([
                knows, state, ans
            ], dim=-1)
            out, h = self.model(input)
        else:
            kc = input[:, 0].long()
            state = input[:, 1:51]
            ans = input[:, 50:51]
            knows = self.embedding(kc)
            input = torch.cat([
                knows, state, ans
            ], dim=-1)
            out, h = self.model(input, h)

        return self.fc2(
                FF.relu(
                    self.fc(out)
                )
            ), h

## Model Training

In [215]:
# utils for multi-lengths

from torch import Tensor

def tensor2list(tensor: Tensor):
    return tensor.cpu().tolist()

def length2mask(length, max_len, valid_mask_val, invalid_mask_val):
    mask = []

    if isinstance(valid_mask_val, Tensor):
        valid_mask_val = tensor2list(valid_mask_val)
    if isinstance(invalid_mask_val, Tensor):
        invalid_mask_val = tensor2list(invalid_mask_val)
    if isinstance(length, Tensor):
        length = tensor2list(length)

    for _len in length:
        mask.append([valid_mask_val] * _len + [invalid_mask_val] * (max_len - _len))

    return torch.tensor(mask)


def get_sequence_mask(shape, sequence_length, axis=1):
    assert axis <= len(shape)
    mask_shape = shape[axis + 1:]

    valid_mask_val = torch.ones(mask_shape)
    invalid_mask_val = torch.zeros(mask_shape)

    max_len = shape[axis]

    return length2mask(sequence_length, max_len, valid_mask_val, invalid_mask_val)


def sequence_mask(tensor: Tensor, sequence_length, axis=1):
    mask = get_sequence_mask(tensor.shape, sequence_length, axis).to(tensor.device)
    return tensor * mask

In [216]:
# padding sequences

from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence

def pad_data(dataset):
    return_set = [torch.Tensor(i[0]) for i in dataset]
    origin_len = [len(i) for i in return_set]
    return_set = pad_sequence(return_set, batch_first=True)
    return return_set, origin_len

In [217]:
def seed_torch(seed=1029):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed) # if you are using multi-GPU.
    torch.backends.cudnn.benchmark = False
    #torch.backends.cudnn.enabled = False
    torch.backends.cudnn.deterministic = True

In [218]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
loss_func = nn.MSELoss(reduction='none')
seed_torch(seed = 20000223)

def eval(eval_loader, eval_len, model):
    losses = []
    for i, batch in enumerate(eval_loader):
        batch = batch.to(device)
        res = model(batch)

        s_idx = i*10
        seq_len = [i-1 for i in eval_len[s_idx:s_idx+10]]
        loss = loss_func(
            res[:, 1:, :],
            batch[:, :-1, 1:51]
        )
        
        loss = torch.mean(sequence_mask(loss, seq_len))
        losses.append(loss.item())
    return np.mean(losses)

# 准备CATE
def prepare_submission(dataset, model):
    cate = []
    for i in range(10):
        seq = dataset[i]
        early = torch.Tensor(seq[0]).unsqueeze(0).to(device)
        interention = torch.Tensor([seq[1]]).to(device)
        ref = torch.Tensor([seq[2]]).to(device)
        target = seq[3]-1
        state_1 = model(early)
        state_1 = state_1[0,-1,:]

        input_2 = torch.cat(
            [interention, state_1, state_1[interention.long()-1]], dim=-1
        )
        input_2 = torch.cat([early, input_2.reshape(1, 1,-1)], dim=1)
        state_2 = model(input_2)
        state_2 = state_2[0,-1,:]
        
        input_3 = torch.cat(
            [ref, state_1, state_1[ref.long()-1]], dim=-1
        )
        input_3 = torch.cat([early, input_3.reshape(1, 1,-1)], dim=1)
        state_3 = model(input_3)
        state_3 = state_3[0,-1,:]
        cate.append((state_3[target] - state_2[target]).item())
    return cate


CATE=[]
EPOCH = [500, 500, 500, 500, 500]
patience = 10
for ex in range(5):
    model = MyRNN().to(device)
    optim = torch.optim.Adam(model.parameters(), lr=0.001)
    # local_dev
    datas = [local_0, local_1, local_2, local_3, local_4]

    train_data = []
    eval_data = []
    
    for n in range(5):
        train_data += datas[n]
    eval_data = datas[ex]

    train_set, train_len = pad_data(train_data)
    eval_set, eval_len = pad_data(eval_data)

    train_loader = DataLoader(train_set, batch_size=10, shuffle=True)
    eval_loader = DataLoader(eval_set, batch_size=10, shuffle=False)

    min_loss=float('inf')
    count=0

    for ep in range(EPOCH[ex]):
        model.train()
        for i, batch in enumerate(train_loader):
            batch = batch.to(device)
            res = model(batch)
            s_idx = i*10
            seq_len = [i-1 for i in train_len[s_idx:s_idx+10]]
            loss = loss_func(
                res[:, 1:, :],
                batch[:, :-1, 1:51]
            )
            loss = torch.mean(sequence_mask(loss, seq_len))
            optim.zero_grad()
            loss.backward()
            optim.step()
        model.eval()
        eval_loss = eval(eval_loader, eval_len, model)

        if eval_loss.item()<min_loss:
            count=0
            best_dict=model.state_dict()
            min_loss=eval_loss.item()
        else:
            count+=1
        if count>=patience:
            model.load_state_dict(best_dict)
            break

        print(f'Train:{ex} Epoch: {ep} Eval_loss: {eval_loss.item()}')
    model.eval()
    CATE.append(prepare_submission(datas[ex], model))

Train:0 Epoch: 0 Eval_loss: 0.06101906672120094
Train:0 Epoch: 1 Eval_loss: 0.025600681081414223
Train:0 Epoch: 2 Eval_loss: 0.02471286989748478
Train:0 Epoch: 3 Eval_loss: 0.015368201769888401
Train:0 Epoch: 4 Eval_loss: 0.013758021406829357
Train:0 Epoch: 5 Eval_loss: 0.012315604835748672
Train:0 Epoch: 6 Eval_loss: 0.010592114180326462
Train:0 Epoch: 7 Eval_loss: 0.009987959638237953
Train:0 Epoch: 8 Eval_loss: 0.008855033665895462
Train:0 Epoch: 9 Eval_loss: 0.008183583617210388
Train:0 Epoch: 10 Eval_loss: 0.008373884484171867
Train:0 Epoch: 11 Eval_loss: 0.007388702593743801
Train:0 Epoch: 12 Eval_loss: 0.0067795137874782085
Train:0 Epoch: 13 Eval_loss: 0.0064430758357048035
Train:0 Epoch: 14 Eval_loss: 0.005730226170271635
Train:0 Epoch: 15 Eval_loss: 0.005419522523880005
Train:0 Epoch: 16 Eval_loss: 0.005258436314761639
Train:0 Epoch: 17 Eval_loss: 0.0050285495817661285
Train:0 Epoch: 18 Eval_loss: 0.004776335321366787
Train:0 Epoch: 19 Eval_loss: 0.0046470267698168755
Train:0 

In [219]:
# local evaluation
res = [np.sqrt(np.mean((label[i] - CATE[i]) ** 2)) for i in range(5)]
print(res)
print(np.mean(res))

[0.10068426058359081, 0.09570050665563447, 0.11524562246133392, 0.13592586389473615, 0.14302208116317588]
0.11811566695169425


In [220]:
# public
seed_torch(seed = 20000223)
CATE=[]
EPOCH = [500, 500, 500, 500, 500]
patience = 10
for ex in range(5):
    model = MyRNN().to(device)
    optim = torch.optim.Adam(model.parameters(), lr=0.001)
    datas = [processed_0, processed_1, processed_2, processed_3, processed_4]

    train_data = []
    eval_data = []
    
    for n in range(5):
        train_data += datas[n]
    eval_data = datas[ex]

    train_set, train_len = pad_data(train_data)
    eval_set, eval_len = pad_data(eval_data)

    train_loader = DataLoader(train_set, batch_size=10, shuffle=True)
    eval_loader = DataLoader(eval_set, batch_size=10, shuffle=False)

    min_loss=float('inf')
    count=0

    for ep in range(EPOCH[ex]):
        model.train()
        for i, batch in enumerate(train_loader):
            batch = batch.to(device)
            res = model(batch)
            s_idx = i*10
            seq_len = [i-1 for i in train_len[s_idx:s_idx+10]]
            loss = loss_func(
                res[:, 1:, :],
                batch[:, :-1, 1:51]
            )
            loss = torch.mean(sequence_mask(loss, seq_len))
            optim.zero_grad()
            loss.backward()
            optim.step()
        model.eval()
        eval_loss = eval(eval_loader, eval_len, model)

        if eval_loss.item()<min_loss:
            count=0
            best_dict=model.state_dict()
            min_loss=eval_loss.item()
        else:
            count+=1
        if count>=patience:
            model.load_state_dict(best_dict)
            break

        print(f'Train:{ex} Epoch: {ep} Eval_loss: {eval_loss.item()}')
    model.eval()
    CATE.append(prepare_submission(datas[ex], model))

Train:0 Epoch: 0 Eval_loss: 0.053027380257844925
Train:0 Epoch: 1 Eval_loss: 0.02729126811027527
Train:0 Epoch: 2 Eval_loss: 0.02048083022236824
Train:0 Epoch: 3 Eval_loss: 0.015378325246274471
Train:0 Epoch: 4 Eval_loss: 0.01484769769012928
Train:0 Epoch: 5 Eval_loss: 0.014867782592773438
Train:0 Epoch: 6 Eval_loss: 0.012649589218199253
Train:0 Epoch: 7 Eval_loss: 0.012052160687744617
Train:0 Epoch: 8 Eval_loss: 0.011289033107459545
Train:0 Epoch: 9 Eval_loss: 0.011446896940469742
Train:0 Epoch: 10 Eval_loss: 0.010604330338537693
Train:0 Epoch: 11 Eval_loss: 0.010643239133059978
Train:0 Epoch: 12 Eval_loss: 0.00964053813368082
Train:0 Epoch: 13 Eval_loss: 0.0096889678388834
Train:0 Epoch: 14 Eval_loss: 0.009406886994838715
Train:0 Epoch: 15 Eval_loss: 0.009196938946843147
Train:0 Epoch: 16 Eval_loss: 0.008272496983408928
Train:0 Epoch: 17 Eval_loss: 0.007810272742062807
Train:0 Epoch: 18 Eval_loss: 0.007265477906912565
Train:0 Epoch: 19 Eval_loss: 0.007224338594824076
Train:0 Epoch: 2

## Result

In [222]:
# save result to cate_estimate.npy
np.save("cate_estimate.npy", CATE)

In [223]:
# private
seed_torch(seed = 20000223)
CATE=[]
EPOCH = [500, 500, 500, 500, 500]
patience = 10
for ex in range(5):
    model = MyRNN().to(device)
    optim = torch.optim.Adam(model.parameters(), lr=0.001)
    datas = [private_0, private_1, private_2, private_3, private_4]

    train_data = []
    eval_data = []
    
    for n in range(5):
        train_data += datas[n]
    eval_data = datas[ex]

    train_set, train_len = pad_data(train_data)
    eval_set, eval_len = pad_data(eval_data)

    train_loader = DataLoader(train_set, batch_size=10, shuffle=True)
    eval_loader = DataLoader(eval_set, batch_size=10, shuffle=False)

    min_loss=float('inf')
    count=0

    for ep in range(EPOCH[ex]):
        model.train()
        for i, batch in enumerate(train_loader):
            batch = batch.to(device)
            res = model(batch)
            s_idx = i*10
            seq_len = [i-1 for i in train_len[s_idx:s_idx+10]]
            loss = loss_func(
                res[:, 1:, :],
                batch[:, :-1, 1:51]
            )
            loss = torch.mean(sequence_mask(loss, seq_len))
            optim.zero_grad()
            loss.backward()
            optim.step()
        model.eval()
        eval_loss = eval(eval_loader, eval_len, model)

        if eval_loss.item()<min_loss:
            count=0
            best_dict=model.state_dict()
            min_loss=eval_loss.item()
        else:
            count+=1
        if count>=patience:
            model.load_state_dict(best_dict)
            break

        print(f'Train:{ex} Epoch: {ep} Eval_loss: {eval_loss.item()}')
    model.eval()
    CATE.append(prepare_submission(datas[ex], model))

Train:0 Epoch: 0 Eval_loss: 0.081059031188488
Train:0 Epoch: 1 Eval_loss: 0.04812378063797951
Train:0 Epoch: 2 Eval_loss: 0.041067443788051605
Train:0 Epoch: 3 Eval_loss: 0.03248113766312599
Train:0 Epoch: 4 Eval_loss: 0.03103226236999035
Train:0 Epoch: 5 Eval_loss: 0.030827386304736137
Train:0 Epoch: 6 Eval_loss: 0.030596664175391197
Train:0 Epoch: 7 Eval_loss: 0.02817377820611
Train:0 Epoch: 8 Eval_loss: 0.027250951156020164
Train:0 Epoch: 9 Eval_loss: 0.027620771899819374
Train:0 Epoch: 10 Eval_loss: 0.023152358829975128
Train:0 Epoch: 11 Eval_loss: 0.020443998277187347
Train:0 Epoch: 12 Eval_loss: 0.016354717314243317
Train:0 Epoch: 13 Eval_loss: 0.014650973491370678
Train:0 Epoch: 14 Eval_loss: 0.013722137548029423
Train:0 Epoch: 15 Eval_loss: 0.01306401938199997
Train:0 Epoch: 16 Eval_loss: 0.012532321736216545
Train:0 Epoch: 17 Eval_loss: 0.011816257610917091
Train:0 Epoch: 18 Eval_loss: 0.010917384177446365
Train:0 Epoch: 19 Eval_loss: 0.010485250502824783
Train:0 Epoch: 20 Eva

In [ ]:
# save result to cate_estimate.npy
np.save("cate_estimate.npy", CATE)

# Note

After running code, the result would save in cate_estimate.npy.